<a href="https://colab.research.google.com/github/omarhashem80/Medical-Cost-Personal/blob/main/insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'insurance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F13720%2F18513%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240218%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240218T172922Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D93032ecb7faf9af5f58e26d9ed6ff5058428f2fce4548a57ffde0f1b866accfcd8c0e1570147e6dda8372e24989d6bcc9dc8e8b263d1793ac841ca85b9609b156bf21ce19d2deb4a5449b7f315e1e1ea16fbd9841303048a6969925a97c874e7731d7c438c1c9794f76e58355a87bc9b7495468869b69dd9da52a571ea338fc1f1dd9161e473685524f33ca30e559decca39f457b50c3634f95793c559131962303985bde1584ed0b76a94a8e0c00a6401c433179627c44872cea89df2b0f4125fb3f5c955189b19a83a86f5591c259ae6f3e43239960793c65efaff32eee7baf01068cdbf5a3acfb1ed196692f6a3d6d18d883f7bf8bf192306018cfb3eda94'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 16425 bytes downloaded
Downloaded and uncompressed: insurance
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import StandardScaler,PolynomialFeatures, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.tree import  DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.pipeline import Pipeline
import plotly.graph_objects as go
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/insurance/insurance.csv


In [3]:
df = pd.read_csv('/kaggle/input/insurance/insurance.csv')
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


## univariante

###num

In [5]:
df.describe(include='number')

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [6]:
px.histogram(df,x='age',text_auto=True)

In [7]:
px.box(df,x='age')

In [8]:
px.histogram(df,x='bmi',text_auto=True)

In [9]:
px.box(df,x='bmi')

In [10]:
px.histogram(df,x='charges',text_auto=True)

In [11]:
px.box(df,x='charges')

In [12]:
px.histogram(df,x='children',text_auto=True)

In [13]:
px.box(df,x='children')

### cat

In [14]:
fig = go.Figure(data=[go.Pie(labels=df['smoker'], hole=0.3,textinfo='percent+label')])
fig.update_traces(textposition='inside')

In [15]:
fig = go.Figure(data=[go.Pie(labels=df['region'], hole=0.3,textinfo='percent+label')])
fig.update_traces(textposition='inside')

In [16]:
fig = go.Figure(data=[go.Pie(labels=df['sex'], hole=0.3,textinfo='percent+label')])
fig.update_traces(textposition='inside')

## bivariante

### num vs num

In [17]:
df.corr()

,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


In [18]:
px.imshow(df.corr(),text_auto='.2f')

In [19]:
px.scatter(df,x='age',y='charges')

In [20]:
px.scatter(df,x='bmi',y='charges')

In [21]:
px.box(df,y='charges', x='children')

In [22]:
df['sex'] = df['sex'].map({'male': 1, 'female': 0})

In [23]:
df['smoker'] = df['smoker'].map({'yes': 1, 'no': 0})

In [24]:
df['region'].value_counts()

southeast    364
southwest    325
northwest    325
northeast    324
Name: region, dtype: int64

In [25]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1].values

In [26]:
 # Construct a column transformer for preprocessing
categorical_features = ['region']  # List of categorical feature names
categorical_transformer = Pipeline(steps=[
    ('onehot_encoder', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)
X = preprocessor.fit_transform(X)
X

array([[ 0.  ,  0.  ,  1.  , ..., 27.9 ,  0.  ,  1.  ],
       [ 0.  ,  1.  ,  0.  , ..., 33.77,  1.  ,  0.  ],
       [ 0.  ,  1.  ,  0.  , ..., 33.  ,  3.  ,  0.  ],
       ...,
       [ 0.  ,  1.  ,  0.  , ..., 36.85,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  1.  , ..., 25.8 ,  0.  ,  0.  ],
       [ 1.  ,  0.  ,  0.  , ..., 29.07,  0.  ,  1.  ]])

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models to evaluate
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Support Vector Machine': SVR(),
    'Decision Tree': DecisionTreeRegressor(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'XGBoost': xgb.XGBRegressor()
}

# Define hyperparameters for each model
params = {
    'Linear Regression': {},
    'Random Forest': {
        'regressor__n_estimators': [100, 200, 300],
        'regressor__max_depth': [None, 5, 10]
    },
    'Support Vector Machine': {'regressor__kernel': ['linear', 'rbf'], 'regressor__C': [0.1, 1, 10]},
    'Decision Tree': {'regressor__max_depth': [None, 5, 10]},
    'K-Nearest Neighbors': {'regressor__n_neighbors': [3, 5, 7]},
    'XGBoost': {
        'regressor__n_estimators': [100, 200, 300],
        'regressor__learning_rate': [0.05, 0.1, 0.2],
        'regressor__max_depth': [3, 4, 5]
    }
}

# Perform model selection and evaluation
results = {}
for model_name, model in models.items():

    # Construct a pipeline with scaler, encoder, and regressor
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', model)
    ])

    print(model_name)
    # Perform hyperparameter tuning with cross-validation
    grid_search = GridSearchCV(pipeline, params[model_name], cv=5, scoring='neg_mean_squared_error', verbose=1)
    grid_search.fit(X_train, y_train)

    # Store results
    results[model_name] = {
        'Best Parameters': grid_search.best_params_,
        'Best Score': -grid_search.best_score_
    }

    # Perform cross-validation to estimate performance
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    results[model_name]['CV Scores'] = -cv_scores  # Convert back to positive MSE

# Print results
for model_name, result in results.items():
    print(f"Model: {model_name}")
    print(f"Best Parameters: {result['Best Parameters']}")
    print(f"Best Score (MSE): {result['Best Score']}")
    print(f"Cross-Validation Scores (MSE): {result['CV Scores']}")
    print(f"Average Cross-Validation Score (MSE): {np.mean(result['CV Scores'])}\n")

# Select the best model based on average cross-validation score
best_model_name = min(results, key=lambda x: np.mean(results[x]['CV Scores']))
best_model = models[best_model_name]

# Evaluate the best model on the testing data
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Testing Data: {mse}")
print(r2_score(y_test, y_pred))

Linear Regression
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Random Forest
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Support Vector Machine
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Decision Tree
Fitting 5 folds for each of 3 candidates, totalling 15 fits
K-Nearest Neighbors
Fitting 5 folds for each of 3 candidates, totalling 15 fits
XGBoost
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Model: Linear Regression
Best Parameters: {}
Best Score (MSE): 37947891.04882816
Cross-Validation Scores (MSE): [40131832.90918392 32077034.53122964 35910188.22467399 46590418.12240703
 35029981.45664619]
Average Cross-Validation Score (MSE): 37947891.04882816

Model: Random Forest
Best Parameters: {'regressor__max_depth': 5, 'regressor__n_estimators': 300}
Best Score (MSE): 21930486.768188074
Cross-Validation Scores (MSE): [25203837.5564116  17124796.11786491 28238061.15195543 29931897.08250165
 25227202.84417288]
Average Cross-Validat

In [28]:
best_model

RandomForestRegressor()

In [29]:
print(r2_score(y_test, y_pred))

0.8615450061505534


In [30]:
ann = Sequential()
ann.add(Dense(8,activation='relu',input_dim=X_train.shape[1]))
ann.add(Dense(16,activation='relu'))
ann.add(Dense(1,activation='linear'))
ann.compile(loss='mean_squared_error',optimizer='Adam')
history = ann.fit(X_train,y_train,epochs=100,validation_split=0.2)
r2_score(y_test,y_pred)

Epoch 1/100
27/27 [==============================] - 1s 10ms/step - loss: 325395680.0000 - val_loss: 310250208.0000
Epoch 2/100
27/27 [==============================] - 0s 3ms/step - loss: 325279552.0000 - val_loss: 310108096.0000
Epoch 3/100
27/27 [==============================] - 0s 3ms/step - loss: 325099936.0000 - val_loss: 309910400.0000
Epoch 4/100
27/27 [==============================] - 0s 3ms/step - loss: 324848736.0000 - val_loss: 309597184.0000
Epoch 5/100
27/27 [==============================] - 0s 4ms/step - loss: 324418528.0000 - val_loss: 309102240.0000
Epoch 6/100
27/27 [==============================] - 0s 3ms/step - loss: 323802432.0000 - val_loss: 308427296.0000
Epoch 7/100
27/27 [==============================] - 0s 4ms/step - loss: 322958720.0000 - val_loss: 307485920.0000
Epoch 8/100
27/27 [==============================] - 0s 3ms/step - loss: 321817376.0000 - val_loss: 306232960.0000
Epoch 9/100
27/27 [==============================] - 0s 3ms/step - loss: 320306

0.8615450061505534